In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib qt5

In [2]:
import sys

In [3]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.PosarMCParameters import *

### Read parameters.xml

In [4]:
data_date = "2018_04_06_14_15_07"
data_dir = "/home/pleroy/DATA/PoSAR-v2_PIMA_TEST-1/" + data_date

In [5]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarMCParameters_v2( params_filename )

In [6]:
samplesPerFile = params.rampsPerFile * params.samplesPerRamp

In [7]:
print(params.rampsPerFile * params.samplesPerRamp * 4)

36000000


### Process timeStamps.data

In [8]:
timeStampsFile = data_dir + "/" + data_date + "_timeStamps.data"
blockNumber, timeStamp = np.loadtxt( timeStampsFile, skiprows = 1, unpack=True )

In [14]:
plt.figure()
plt.suptitle(data_date)

plt.subplot(211)
plt.plot( blockNumber, timeStamp, '.', label="timeStamps" )
plt.grid()
plt.legend()

plt.subplot(212)
plt.plot( np.diff(timeStamp[1:]/1000), '.', label="diff timeStamps" )
plt.grid()
plt.legend()

### Read data

In [10]:
firstFileIsFull = 1

In [11]:
buffer_a = 0
filename_a = data_dir + '/record' + str(buffer_a) + '.bin'
fd_a = open(filename_a,'rb')
if firstFileIsFull == 1:
    rampsToRead = params.rampsPerFile
else:
    rampsToRead = int(params.rampsPerFile / 2)
record_a = np.fromfile(fd_a, dtype = np.uint16, count = rampsToRead * params.samplesPerRamp * 2)
record_a_0 = record_a[::2].reshape(rampsToRead, params.samplesPerRamp)
record_a_1 = record_a[1::2].reshape(rampsToRead, params.samplesPerRamp)

buffer_b = 366
filename_b = data_dir + '/record' + str(buffer_b) + '.bin'
fd_b = open(filename_b,'rb')
record_b = np.fromfile(fd_b, dtype = np.uint16, count = params.rampsPerFile * params.samplesPerRamp * 2)
record_b_0 = record_b[::2].reshape(params.rampsPerFile, params.samplesPerRamp)
record_b_1 = record_b[1::2].reshape(params.rampsPerFile, params.samplesPerRamp)

In [12]:
data_reshaped_a = record_a_0.reshape(rampsToRead, params.samplesPerRamp)
data_reshaped_b = record_b_0.reshape(params.rampsPerFile, params.samplesPerRamp)

In [13]:
scalingFactor = 2 / 65535
offset = -1

In [14]:
data_scaled_a = data_reshaped_a * scalingFactor + offset
data_scaled_b = data_reshaped_b * scalingFactor + offset

### Plot data

In [15]:
font = { 'family' : 'sans-serif',
        'weight' : 'bold',
        'size'   : 12}

matplotlib.rc('font', **font)

In [17]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(data_reshaped_a[0,:], color='b', label = str(buffer_a) + ' ' + data_date)
ax1.plot(data_reshaped_b[0,:], color='g', label = str(buffer_b) + ' ' + data_date)
ax1.set_ylabel('raw')
ax1.legend()

ax2.plot(data_scaled_a[0,:], color='b', label = str(buffer_a) + ' ' + data_date)
ax2.plot(data_scaled_b[0,:], color='g', label = str(buffer_b) + ' ' + data_date)
ax2.set_ylabel('scaled [V]')
    
ax1.grid()

### Plot spectrum

In [18]:
nbSamplesPerHalfRamp = int(params.samplesPerRamp/2)

In [25]:
firstRamp = data_reshaped_a[0, 0:nbSamplesPerHalfRamp]
firstRampScaled = firstRamp * scalingFactor + offset
fft = np.fft.fft(firstRampScaled)

firstRamp_b = data_reshaped_b[0, 0:nbSamplesPerHalfRamp]
firstRampScaled_b = firstRamp_b * scalingFactor + offset
fft_b = np.fft.fft(firstRampScaled_b)

In [26]:
plt.figure()
plt.plot(firstRampScaled, label=data_date)
plt.plot(firstRampScaled_b, label=data_date)
plt.legend()
plt.grid()

In [27]:
max_fft = 20 * np.log10( np.abs( np.max(np.abs(fft)) ) )
print("max fft = {:.2f} dB".format(max_fft))

max fft = 35.51 dB


In [29]:
plt.figure()
plt.plot(20*np.log10(np.abs(fft)), label=data_date)
plt.plot(20*np.log10(np.abs(fft_b)), label=data_date)
plt.grid()
plt.legend()